In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import itertools

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
leaderboard = {}

# Load the data
Load matrix and create list of samples and entries

In [1]:
df = pandas.read_csv("results/reannotated_simfusions_binmx.tsv", sep="\t", index_col=0)
df.columns = df.columns.map(lambda x:x.replace(".", "|"))
samples = list(set(i.split("|")[0] for i in df.columns if i not in ['fusion_key', 'fusion_id']))
entries = list(set(i.split("|")[1] for i in df.columns if i not in ['fusion_key', 'fusion_id']))
#samples.remove("giab")
entries.remove("truth")

NameError: name 'pandas' is not defined

In [4]:
def getConfMatrix(df, entries, sample, truth):
    o = {}
    for e in entries:
        k = "%s|%s" % (sample, e)
        if k in df:
            pos = df[df[k] > 0].index
            tp = sum(pos.isin(truth))
            fp = sum(~pos.isin(truth))
            fn = sum(~truth.isin(pos))
            o[e] = {'tp' : tp, 'fp' : fp, 'fn' : fn}
    return o

In [5]:
def f1Score(m):
    return (2.0 * (m['tp'])) / (2.0*m['tp'] + m['fp'] + m['fn'])

In [6]:
def tprScore(m):
    return (m['tp']) / (m['tp'] + m['fn'])

In [7]:
def fdrScore(m):
    return (m['fp']) / (m['fp'] + m['tp'])

# Create overall senstivity scores

In [8]:
tpr_data = {}
fdr_data = {}
f1_data = {}
count_data = {}
tp_data = {}
call_data = {}
for s in samples:
    td = {}
    fd = {}
    t = df[ df["%s|truth" % (s)] == 1 ].index
    res = getConfMatrix(df, entries, s, t)
    tpr_data[s] = dict( (a[0], tprScore(a[1])) for a in res.items() )
    fdr_data[s] = dict( (a[0], fdrScore(a[1])) for a in res.items() )
    f1_data[s] = dict( (a[0], f1Score(a[1])) for a in res.items() )
    tp_data[s] = dict( (a[0], a[1]['tp']) for a in res.items() )
    call_data[s] = dict( (a[0], a[1]['tp'] + a[1]['fp']) for a in res.items() )
    count_data[s] = len(t)
tpr_df = pandas.DataFrame(tpr_data)
fdr_df = pandas.DataFrame(fdr_data)
f1_df = pandas.DataFrame(f1_data)
tp_df = pandas.DataFrame(tp_data)
call_df = pandas.DataFrame(call_data)
spike_count = pandas.Series(count_data)

In [9]:
leaderboard['sim_tpr'] = tpr_df.sum(axis=1) / len(samples)

In [10]:
leaderboard['sim_fdr'] = fdr_df.sum(axis=1) / len(samples)

In [11]:
leaderboard['sim_f1'] = f1_df.sum(axis=1) / len(samples)

In [12]:
leaderboard['sim_call_count'] = call_df.sum(axis=1)

In [13]:
leaderboard['sim_tp_count'] = tp_df.sum(axis=1)

In [14]:
def entryColumns(a, df):
    c = []
    for b in samples:
        k = "%s|%s" % (b, a)
        if k in df.columns:
            c.append(k)
    return c

In [15]:
data = {}
for i in entries:
    for j in entries:
        a = df[entryColumns(i, df)].sum(axis=1)
        b = df[entryColumns(j, df)].sum(axis=1)
        c = a.corr(b)
        data[i] = {**data.get(i, {}), **{j:c}}
        data[j] = {**data.get(j, {}), **{i:c}}
corrDF = pandas.DataFrame(data)

In [16]:
clines = {}
for i in samples:
    if i != 'giab':
        cell = i.split("_")[0]
        if cell in clines:
            clines[cell].append(i)
        else:
            clines[cell] = [i]

In [17]:
true_sites = set()
for c in df.columns:
    if c.endswith("|truth"):
        true_sites.update(set(df[df[c] > 0][c].index))

In [18]:
pandas.DataFrame(leaderboard).rank()

,sim_tpr,sim_fdr,sim_f1,sim_call_count,sim_tp_count
7185743,33.0,19.0,33.0,25.5,32.5
7238196,17.0,26.0,16.0,31.0,16.5
7252855,8.0,22.0,9.0,3.0,8.0
7269246,2.0,33.0,2.0,32.0,2.0
7378934,32.0,21.0,32.0,29.0,32.5
7488045,5.0,29.0,5.0,2.0,5.0
7997358,16.0,16.0,17.0,16.0,16.5
8040408,31.0,11.0,30.0,21.0,31.0
8050015,3.0,27.0,3.0,5.0,3.0
8071387,13.0,25.0,13.0,28.0,13.0


In [19]:
leadDF = pandas.DataFrame(leaderboard) 

In [20]:
decending = ["sim_tpr", "sim_f1"]
acending = ["sim_fdr"]
leadRankDF = leadDF[decending].join(1.0 - leadDF[acending]).rank(ascending=False).rename(columns=lambda x: x+"_rank")

In [21]:
leadDF.join(leadRankDF).to_csv("fusion_sim_leaderboard.tsv", sep="\t")

In [22]:
leadDF.join(leadRankDF)

,sim_tpr,sim_fdr,sim_f1,sim_call_count,sim_tp_count,sim_tpr_rank,sim_f1_rank,sim_fdr_rank
7185743,0.613757,0.277006,0.651295,691.0,466.0,1.0,1.0,19.0
7238196,0.506357,0.530524,0.461929,824.0,393.0,17.0,18.0,26.0
7252855,0.092933,0.361390,0.079881,297.0,58.0,26.0,25.0,22.0
7269246,0.001143,0.999355,0.000825,1042.0,1.0,32.0,32.0,33.0
7378934,0.611083,0.321656,0.608871,735.0,466.0,2.0,2.0,21.0
7488045,0.022070,0.723864,0.033521,242.0,22.0,29.0,29.0,29.0
7997358,0.482781,0.184728,0.515394,627.0,393.0,18.0,17.0,16.0
8040408,0.578130,0.168949,0.598353,656.0,452.0,3.0,4.0,11.0
8050015,0.007880,0.708189,0.009289,466.0,6.0,31.0,31.0,27.0
8071387,0.346682,0.495144,0.311324,703.0,279.0,21.0,21.0,25.0
